In [1]:
import socket
import time
import csv
import random

In [2]:
API_dataset_csv_path = "/home/ubuntu/jupyter/Datasets/goodreads_API_dataset.csv"
scraped_dataset_csv_path = "/home/ubuntu/jupyter/Datasets/goodreads_scraped_dataset.csv"

## here i collect all the paths
#dataset_paths = [API_dataset_csv_path, scraped_dataset_csv_path]
dataset_paths = [API_dataset_csv_path]

In [3]:
# this function is responsible to write the dataset on the socket line by line so that spark
# reads it on the other end.
# the line is a touple from a randoom table

def sendData(connection, dataset):

    print("Start sending data")
    count = 0

    # Open file
    with open(dataset) as file_obj:
        # Create reader object by passing the file
        # object to reader method

        heading=next(file_obj)

        reader_obj = csv.reader(file_obj)

        # Iterate over each row in the csv
        # file using reader object
        for line in reader_obj:
            #print(line)

            try:

                line_string = f'{str(line)}\n'
                byte_message = line_string.encode('utf-8')
                print(f'Debug byte_message {byte_message}')
                connection.sendall(byte_message)
                time.sleep(random.uniform(0, 0.5))  # Aggiungi un delay per non sovraccaricare il ricevitore
                count += 1
                print("Sent:", line_string)
            except Exception as e:
                print("Error sending data:", e)
                break

In [4]:
for path in dataset_paths:
    host = "localhost"
    port = 9999
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((host, port))
    s.listen(5)
    print("Server is listening on", port)

    try:
        connection, addr = s.accept()
        print("Connected by", addr)
        sendData(connection, path)
    except Exception as e:
        print("Connection error:", e)
    finally:
        connection.close()

Server is listening on 9999
Connected by ('127.0.0.1', 40070)
Start sending data
Debug byte_message b"['1', 'Harry Potter and the Half-Blood Prince (Harry Potter  #6)', 'J.K. Rowling/Mary GrandPr\xc3\xa9', '4.57', '0439785960', '9780439785969', 'eng', '652', '2095690', '27591', '9/16/2006', 'Scholastic Inc.']\n"
Sent: ['1', 'Harry Potter and the Half-Blood Prince (Harry Potter  #6)', 'J.K. Rowling/Mary GrandPré', '4.57', '0439785960', '9780439785969', 'eng', '652', '2095690', '27591', '9/16/2006', 'Scholastic Inc.']

Debug byte_message b"['2', 'Harry Potter and the Order of the Phoenix (Harry Potter  #5)', 'J.K. Rowling/Mary GrandPr\xc3\xa9', '4.49', '0439358078', '9780439358071', 'eng', '870', '2153167', '29221', '9/1/2004', 'Scholastic Inc.']\n"
Sent: ['2', 'Harry Potter and the Order of the Phoenix (Harry Potter  #5)', 'J.K. Rowling/Mary GrandPré', '4.49', '0439358078', '9780439358071', 'eng', '870', '2153167', '29221', '9/1/2004', 'Scholastic Inc.']

Debug byte_message b"['4', 'Har

KeyboardInterrupt: 